# Final Project
## Remi LeBlanc and Max Shinnerl

https://github.com/fastai/fastbook/blob/master/12_nlp_dive.ipynb

Some info:

Original fastai model was pretrained with wiki and finetuned on these african new articles
Task to was a binary classification to determine in the tags 'Agribusiness' or 'Food and Agriculture' were in the list of tags for each article. One problem with this was only 3% of the articles had this tag, this made it a bit interesting. Accuracy is not a good metric, used f1/f beta score and precision and recall as our metric.
Fastai model performed about 80% recall 60% precision.

For our project we can either continue looking for agriculture news articles, becasue then we wont have to do anything with the fastai notebooks, or we can do a multilabel or multiclass somehow. 

The link above in from the fastai text book and he goes through out to make a lanugage model from scratch in pytorch - we might be able to follow that

### How will we compare the models?

We can just compare the metrics we chose - precision/recall and f1score.
We can talk about how it is harder to build from scratch, but have more flexibility. We should find something to do in the pytorch model that helps our model we couldnt do in fastai. Like perhaps with our unbalanced data

In [21]:
# output_0628_fixedwithspace_updated.json has about 100,000 articles

In [29]:
# output_0728.json has about 500k with 241k in english

In [20]:
import json
import pandas as pd
import os

In [23]:
#with open('output_0628_fixedwithspace_updated.json') as json_file:
with open('output_0728.json') as json_file:

    data = json.load(json_file)

In [24]:
len(data)

448905

In [25]:
all_articles = {key:val for key, val in data.items() if type(val) == dict}


In [26]:
len(all_articles)

448905

In [27]:
all_articles = {k:v for k,v in all_articles.items() if v!='Article NA'}
all_articles = {k:v for k,v in all_articles.items() if v['language']=='English'}

In [28]:
len(all_articles)

241202

In [30]:
all_articles['202003010001']

{'title': 'Somalia: Sufi Muslim Leaders Surrender to Government',
 'article': ' The leaders of a Sufi Muslim group turned themselves into the custody of the Somali government Saturday after fighting left 22 people dead in central Somalia. Moallim Mohamud Sheikh, the spiritual leader, and Sheikh Mohamed Shakir, the chief of Ahlu-Sunna Wal-Jamaa (ASWJ), are in the custody of the Somali national army in the town of Dhusamareb after the group’s militias were overpowered in a battle with government forces. Dhusamareb is the administrative capital of Galmudug state. "Our security forces have ended the standoff and disarmed all ASWJ militias,” Osman Isse Nur, the spokesperson of the newly elected president, told VOA. Speaking in a video posted online, ASWJ chief Sheikh Shakir said his group ceded power to the Somali national army. "We agreed to end the fighting for the sake of the civilians. We agreed to hand over ASWJ militias to the commander general who will, in return, take responsibility

In [31]:
# Clean up non-matching column names
for k,v in all_articles.items():
    # Update tag to tags
    t = v.get('tag')
    if t:
        all_articles[k]['tags'] = t
        del all_articles[k]['tag']
        
    # Update full_story to full
    f = v.get('full text')
    if str(f)!='None':
        all_articles[k]['full_story'] = f
        del all_articles[k]['full text']

In [32]:
df = pd.DataFrame.from_dict(all_articles,orient='index')
df[df['full text'].notnull()]

,title,article,author,date,source,language,original_url,tags,full_story,full text,country


In [33]:
df = df.drop(['full text'],axis=1)
len(df)

241202

In [34]:
df['liststring'] = [','.join(map(str, l)) for l in df['tags']]

In [35]:
# Get the indices of the articles with agri tags
indices = []
for i, tags in enumerate(df['liststring']):
    if 'Agribusiness' in tags or 'Food and Agriculture' in tags:
        indices.append(i)
# make column of 0 and 1 in df for appearance of that tag
ies = [0]*df.shape[0]
for x in indices:
    ies[x]+=1
df['agri_label'] = ies

In [36]:
df['agri_label'] = ies

In [37]:
# 3% of articles have agri/foodag tag
sum(df['agri_label']==1), len(df), sum(df['agri_label']==1)/len(df)

(7189, 241202, 0.0298048938234343)

In [38]:
df.head()

,title,article,author,date,source,language,original_url,tags,full_story,country,liststring,agri_label
202003010001,Somalia: Sufi Muslim Leaders Surrender to Gove...,The leaders of a Sufi Muslim group turned the...,By Abdulaziz Osman,1 March 2020,"Voice of America (Washington, DC)",English,https://www.voanews.com/africa/somalias-sufi-m...,"[Somalia, East Africa, Legal Affairs, Conflict...",False,NaN,"Somalia,East Africa,Legal Affairs,Conflict,Arm...",0
202003010002,Libya: UN-Mediated Political Talks on Libya En...,U.N.-mediated political talks aimed at resolv...,By Lisa Schlein,1 March 2020,"Voice of America (Washington, DC)",English,https://www.voanews.com/middle-east/un-mediate...,"[Libya, Conflict, External Relations, North Af...",False,NaN,"Libya,Conflict,External Relations,North Africa...",0
202003010003,Nigeria: Obasanjo At 83 - a Leader and His Cou...,"""I don't want to be remembered. I am still he...",N/A,1 March 2020,This Day (Lagos),English,https://www.thisdaylive.com/index.php/2020/03/...,"[Nigeria, West Africa, Governance]",False,NaN,"Nigeria,West Africa,Governance",0
202003010004,"Sudan: Service of 14 Ambassadors, 35 Diplomats...",Khartoum — The Higher Committee for the disma...,N/A,29 February 2020,Sudan News Agency (Khartoum),English,https://suna-sd.net/en/single?id=561889,"[Sudan, East Africa, Governance]",False,NaN,"Sudan,East Africa,Governance",0
202003010005,Sudan: German President Ends Sudan Visit With ...,Khartoum — German President Frank-Walter Stei...,N/A,29 February 2020,Radio Dabanga (Amsterdam),English,https://www.dabangasudan.org/en/all-news/artic...,"[Sudan, External Relations, East Africa, Gover...",False,NaN,"Sudan,External Relations,East Africa,Governanc...",0


 We will build a language model from the 'article' column to predict the 'agri_label' column or the 'tags' column